In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import dspy

lm = dspy.LM("ollama_chat/qwen2.5", api_base="http://localhost:11434", api_key="")
dspy.configure(lm=lm)

Test LM

In [3]:
lm(messages=[{"role": "user", "content": "Hi!"}])  

['Hello! How can I assist you today?']

File ingestion

In [ ]:
import os

if not os.path.exists("../docs"):
    os.makedirs("../docs")

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.embed_model = OllamaEmbedding(model_name="mxbai-embed-large")
docs = SimpleDirectoryReader()

# Declare!
Not prompt

## Chain Of Thought